# ACSE-7 (Inversion and Optimisation)  <a class="tocSkip"></a>

# Homework Lecture 7: PDE-Constrained Optimisation (solutions) <a class="tocSkip"></a>

In [1]:
import numpy as np
import scipy.linalg as sl
import matplotlib.pyplot as plt
# font sizes for plots
plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Dejavu Sans']

In [2]:
%matplotlib inline

# Diffusion Equation

We consider the diffusion equation

$$
  \frac{\partial u}{\partial t} - \kappa \frac{\partial^2}{\partial x^2} = s(x, t)
$$

which has many uses, such as the dispersal of microparticles 
through random motion (Brownian motion), the spread of a disolved substance, e.g. a pollutant, in a fluid, or the spread of heat. For the moment let's think of $u$ as a concentration. The coefficient $\kappa$ is the diffusion coefficient. We also consider a source term $s(x,t)$ which represent release of the concentration (can be a sink as well when negative).

This PDE comes with the following initial and boundary conditions:

$$
  u(x, 0) = u_{\text{ic}}(x), \text{ and } \\
  \kappa\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_0} = \Phi_0 \text{ and }
  \kappa\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_1} = \Phi_1
$$

Note that unlike for the advection equation example in the lecture notes we need a boundary condition on both sides of the domain which we assume is the interval $[x_0, x_1]$, and here we chose to apply a Neumann condition. The boundary values $\Phi_{0,1}$ represent the diffusive flux through both boundaries.

# Tangent linear approach

* we have observation data that tells us what the concentration $u(x,t)$ should be at $t=T$, say $u(x, T) \approx u_{\text{obs}}(x)$. We want to chance some parameters $m$ in our diffusion model to improve the match between the predicted concentration $u(x,t)$ predicted by the model and the observation. Write down what a suitable functional $f(u, m)$ would be for the associated optimisation problem

### Answer <a class="tocSkip"></a>
This is the same as for the advection example in our lecture notes, namely we take the (square of the) L2-norm of the mismatch:

$$
  f(u, m) = \int_0^L (u(x,T)-u_{\text{obs}}(x))^2 \mathrm{d}x
$$


* First we consider optimising the diffusion coefficient, so we choose $m$ to be a single parameter, $m=\kappa$, that we want to optimize to improve the match with observations. Write the PDE system as a constraint $g(u, m)$ similar to the advection example with an entry for the actual PDE, and entries for the initial and both boundary conditions.

### Answer <a class="tocSkip"></a>

$$
  g(u, m) = \begin{pmatrix}  
  \frac{\partial u}{\partial t} - m \frac{\partial^2 u}{\partial x^2} - s(x, t) \\
  u(\cdot, 0) - u_{\text{ic}}(\cdot) \\
  \kappa\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_0} - \Phi_0 \\
  \kappa\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_1} - \Phi_1
  \end{pmatrix} =
  \begin{pmatrix}
    0 \\ 0 \\ 0 \\0
  \end{pmatrix}
  \;\;\;
  \begin{array}{c}
    \text{PDE constraint} \\ \text{initial condition} \\
    \text{boundary condition at $x_0$} \\
    \text{boundary condition at $x_1$}
  \end{array}
$$

* In the lecture notes we saw that in the tangent linear approach we need to solve

$$
  \frac{\partial g}{\partial u}\delta u = -\frac{\partial g}{\partial m}\delta m
$$

to obtain the perturbation $\delta u$ in $u$ due to a perturbation $\delta m$ in $m$. In other words:

$$
  \delta u = \frac{du}{dm} \delta m
$$

Work out what the PDE is for $\delta u$, including initial and boundary conditions.

### Answer <a class="tocSkip"></a>
We have

$$
  \frac{\partial g(u, m)}{\partial u} \delta u = \begin{pmatrix}  
  \frac{\partial \delta u}{\partial t} - m\frac{\partial^2 \delta u}{\partial x^2} \\
  \delta u(\cdot, 0) \\
  m\left.\frac{\partial \delta u(x,t)}{\partial x}\right|_{x=x_0}  \\
  m\left.\frac{\partial \delta u(x,t)}{\partial x}\right|_{x=x_1}
  \end{pmatrix}
  \quad \text{ and } \quad
  \frac{\partial g(u, m)}{\partial m} \delta m = \begin{pmatrix}  
  -\delta m\frac{\partial^2 u}{\partial x^2} \\
  0 \\
  \delta m\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_0} \\
  \delta m\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_1}
  \end{pmatrix}
$$

As a PDE that becomes:

$$
  \frac{\partial \delta u}{\partial t} - m\frac{\partial^2 \delta u}{\partial x^2} 
  = \delta m\frac{\partial^2 u}{\partial x^2} 
$$

with initial and boundary conditions:

$$
  \delta u(x, 0) = 0, \text{ and } \\
  m\left.\frac{\partial\delta u(x,t)}{\partial x}\right|_{x=x_0} = 
  \delta m\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_0}
  \quad \text{ and } \quad
  m\left.\frac{\partial\delta u(x,t)}{\partial x}\right|_{x=x_1} = 
  \delta m\left.\frac{\partial u(x,t)}{\partial x}\right|_{x=x_1}
$$

* Now, instead of the diffusivity coefficient, we want to invert for optimal boundary fluxes $\Phi_1$ and $\Phi_2$. We want to find the optimal *combination* of these two values, so we optimize for them simultaneously with $m=(\Phi_1, \Phi_2)$ now being a 2-vector. What is the consequence of this for the tangent-linear approach? You don't need to work out the new equations exactly (although it's a good exercise), just indicate what the main difference is with the previous case.

### Answer <a class="tocSkip"></a>
The short answer is that because we now have two optimisation parameters, we need to solve 
two separate PDEs for $\delta u_1$ and a $\delta u_2$:
    
$$
  \delta u_1 = \frac{du}{dm_1} \delta m_1,
  \delta u_2 = \frac{du}{dm_2} \delta m_2,
$$

where $\delta m_1$ and $\delta m_2$ are the pertubations in $m_1=\Phi_1$ and $m_2=\Phi_2$ respectively.


The left-hand side for $\delta u_1$ and $\delta u_2$ look the same:

$$
  \frac{\partial g(u, m)}{\partial u} \delta u_{1,2} = \begin{pmatrix}  
  \frac{\partial \delta u_{1,2}}{\partial t} - \kappa\frac{\partial^2 \delta u_{1,2}}{\partial x^2} \\
  \delta u_{1,2}(\cdot, 0) \\
  \kappa\left.\frac{\partial \delta u_{1,2}(x,t)}{\partial x}\right|_{x=x_0}  \\
  \kappa\left.\frac{\partial \delta u_{1,2}(x,t)}{\partial x}\right|_{x=x_1}
  \end{pmatrix}
$$

but on the right-hand side we have

$$
  \frac{\partial g(u, m)}{\partial m_1} \delta m_1 = \begin{pmatrix}  
  0 \\
  0 \\
  -\delta m_1 \\
  0
  \end{pmatrix}
  \quad\text{ and }\quad
  \frac{\partial g(u, m)}{\partial m_1} \delta m_1 = \begin{pmatrix}  
  0 \\
  0 \\
  0 \\
  -\delta m_2
  \end{pmatrix}
$$

which results in the same PDE for both $\delta u_1$ and $\delta u_2$:

$$
  \frac{\partial \delta u_{1,2}}{\partial t} - \kappa\frac{\partial^2 \delta u_{1,2}}{\partial x^2} = 0
$$

with initial condition

$$
  \delta u_{1,2}(x,0) = 0
$$

but with different boundary conditions:

$$
  \kappa\left.\frac{\partial \delta u_1(x,t)}{\partial x}\right|_{x=x_0} = \delta m_1, \quad
  \kappa\left.\frac{\partial \delta u_1(x,t)}{\partial x}\right|_{x=x_1} = 0
$$

for $\delta u_1$, whereas for $\delta u_2$ we have:

$$
  \kappa\left.\frac{\partial \delta u_2(x,t)}{\partial x}\right|_{x=x_0} = 0, \quad
  \kappa\left.\frac{\partial \delta u_2(x,t)}{\partial x}\right|_{x=x_1} = \delta m_2
$$

* Finally, we consider optimising the source term $s(x,t)$ and thus the optimisation parameter $m$ becomes a function $m(x,t) = s(x,t)$. Would we still be able to use the tangent-linear approach in this case to compute the gradient $df/dm$ ?

### Answer <a class="tocSkip"></a>
No, the tangent-linear approach would not be practically possible. We can solve the perturbation $\delta u$ in $u$ for any perturbation $\delta m=\delta s$, but since there are infinitely many possibilities for the source perturbation, we would never be able to compute the complete gradient $df/dm$ in this way.